In [24]:
from pymongo import MongoClient
from bson import ObjectId
import json
import openai
import os
from dotenv import load_dotenv
import gradio as gr
from qdrant_client import QdrantClient


In [25]:

mongoDBClient = MongoClient("mongodb://localhost:27017/")
db = client['clearml']
collection = db['crawled_data']
client = QdrantClient(host="localhost", port=6333)


In [13]:

load_dotenv()

openai.api_key = os.getenv('OPEN_AI_API_KEY')

def generate_embeddings(text):
    print(text)
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding


In [18]:
def RAG_Context (query):
    query_embedding = generate_embeddings(query)
    results = client.search(
    collection_name="robotics-documents",
    query_vector=query_embedding,
    limit=3
    )
    context = "\n\n".join([f"Title: {match.payload['title']}\n{match.payload['content']}" for match in results])
    return context
    

In [17]:
def generateResponse(query, context):
    response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # Use "gpt-4" if you want a more advanced model
    messages=[
        {"role": "system", "content": "You are a helpful assistant for answering questions based on provided context."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"}
    ],
    max_tokens=200,
    temperature=0  # Adjust temperature for deterministic responses
)
    return response.choices[0].message.content


In [30]:
import gradio as gr

# Prefilled suggestions for the dropdown
suggestions = [
    "Tell me how can I navigate to a specific pose - include replanning aspects in your answer.",
    "What is ROS2",
    "What are the three different communication channels?"
]

# Function to handle the query
def getResp(query):
    context = RAG_Context(query)
    resp = generateResponse(query, context)
    print(resp)
    return resp

# Function to handle dropdown selection and refill the text box
def update_text(selected):
    return selected  # The selected suggestion replaces the text box content

# Gradio interface setup
with gr.Blocks() as demo:
    with gr.Row():
        dropdown = gr.Dropdown(
            choices=suggestions, 
            label="Suggestions", 
            interactive=True
        )
        textbox = gr.Textbox(
            placeholder="Type your query here...",
            label="Your Query",
            lines=2
        )
    # Update textbox when dropdown changes
    dropdown.change(update_text, inputs=[dropdown], outputs=[textbox])
    submit_button = gr.Button("Submit")
    response_box = gr.Textbox(label="Response")

    # Handle submission
    submit_button.click(getResp, inputs=[textbox], outputs=[response_box])

demo.launch()


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Tell me how can I navigate to a specific pose - include replanning aspects in your answer.
To navigate to a specific pose in ROS 2 with the Nav2 framework, you can follow these steps:

1. **Set Up the Environment**:
   - Ensure you have your robot model and world set up in Gazebo.
   - Launch the Nav2 infrastructure, including the necessary components like AMCL for localization.

2. **Create a Behavior Tree**:
   - Define a behavior tree that includes nodes for moving the robot to different poses.
   - Each node in the behavior tree corresponds to a specific location where the robot should navigate.

3. **Implement the Behavior Tree in C++**:
   - Create a C++ package for your application layer, such as `tb3_autonomy`.
   - Implement the behavior tree logic in this package using Behavior Tree CPP Library.

4. **Launch the Autonomy Node**:
   - Create a launch file for launching the autonomy node, which will execute the behavior tree logic.
   - Include the necessary parameters for spec

In [9]:
query = "What are the three different communication channels?"
query_embedding = generate_embeddings(query)

results = client.search(
    collection_name="robotics-documents",
    query_vector=query_embedding,
    limit=3
)

for match in results:
    print(f"Title: {match.payload['title']}")
    print(f"Content: {match.payload['content']}")
    print(f"Platform: {match.payload['platform']}")
    print(f"Link: {match.payload['link']}\n")

NameError: name 'generate_embeddings' is not defined